Identifying deviations in responses for the same nationality using different locales.

In [1]:
import pandas as pd

df_responses_gpt = pd.read_csv('responses_parsed_gpt.csv', sep=';', encoding='utf-8-sig')

Descriptive stats for getting min/max for each model.

In [2]:
desc = df_responses_gpt[['question','nationality','local','score_gpt-4o','score_gpt-3.5-turbo','score_yandexgpt']].groupby(['nationality','question','local']).describe()

desc

score_gpt-4o                                     \
                                  count      mean       std  min  25%  50%   
nationality question local                                                   
American    1        EN            10.0  1.350000  0.474342  1.0  1.0  1.0   
                     RU            10.0  1.500000  0.000000  1.5  1.5  1.5   
            2        EN            10.0  1.900000  0.210819  1.5  2.0  2.0   
                     RU            10.0  1.650000  0.337474  1.0  1.5  1.5   
            3        EN            10.0  2.000000  0.000000  2.0  2.0  2.0   
...                                 ...       ...       ...  ...  ...  ...   
Russian     22       RU             9.0  1.833333  0.250000  1.5  1.5  2.0   
            23       EN            10.0  1.500000  0.000000  1.5  1.5  1.5   
                     RU             8.0  1.625000  0.231455  1.5  1.5  1.5   
            24       EN             9.0  3.055556  0.916667  1.5  2.5  3.0   
                     RU             5.0  2.200000  0.447214  2.0  2.0  2.0   

                                       score_gpt-3.5-turbo            ...  \
                              75%  max               count      mean  ...   
nationality question local                                            ...   
American    1        EN     1.875  2.0                10.0  1.700000  ...   
                     RU     1.500  1.5                10.0  2.000000  ...   
            2        EN     2.000  2.0                10.0  1.800000  ...   
                     RU     2.000  2.0                10.0  1.350000  ...   
            3        EN     2.000  2.0                10.0  2.000000  ...   
...                           ...  ...                 ...       ...  ...   
Russian     22       RU     2.000  2.0                10.0  1.800000  ...   
            23       EN     1.500  1.5                10.0  1.000000  ...   
                     RU     1.625  2.0                10.0  1.900000  ...   
            24       EN     4.000  4.0                 9.0  1.111111  ...   
                     RU     2.000  3.0                 8.0  2.062500  ...   

                                       score_yandexgpt                      \
                              75%  max           count      mean       std   
nationality question local                                                   
American    1        EN     2.000  2.0             9.0  1.833333  0.433013   
                     RU     2.000  2.0            10.0  1.850000  0.241523   
            2        EN     2.000  2.0            10.0  2.000000  0.000000   
                     RU     1.875  2.0            10.0  1.950000  0.158114   
            3        EN     2.000  2.0            10.0  2.000000  0.000000   
...                           ...  ...             ...       ...       ...   
Russian     22       RU     2.000  2.0             7.0  1.928571  0.188982   
            23       EN     1.000  1.0             9.0  2.111111  0.857969   
                     RU     2.000  4.0             3.0  2.750000  1.089725   
            24       EN     1.000  2.0             7.0  2.285714  0.859125   
                     RU     2.000  2.5             1.0  2.000000       NaN   

                                                          
                            min    25%   50%    75%  max  
nationality question local                                
American    1        EN     1.5  1.500  1.50  2.000  2.5  
                     RU     1.5  1.625  2.00  2.000  2.0  
            2        EN     2.0  2.000  2.00  2.000  2.0  
                     RU     1.5  2.000  2.00  2.000  2.0  
            3        EN     2.0  2.000  2.00  2.000  2.0  
...                         ...    ...   ...    ...  ...  
Russian     22       RU     1.5  2.000  2.00  2.000  2.0  
            23       EN     1.5  1.500  1.50  2.500  4.0  
                     RU     2.0  2.125  2.25  3.125  4.0  
            24       EN     1.5  1.500  2.00  3.000  3

In [3]:
models = ['score_gpt-4o', 'score_gpt-3.5-turbo', 'score_yandexgpt']

for model in models:
    desc[(model, 'range')] = desc[(model, 'max')] - desc[(model, 'min')]

    desc[[(model, 'mean'),(model, 'std'),(model, 'min'),(model, 'max'),(model, 'range')]][desc[(model, 'range')] >= 2.0].apply(lambda x: round(x,2)).to_csv(f'responses_range_{model}.csv', sep=';', encoding='utf-8-sig')

Defining as outliers cases where difference in scores 2.0 or more between max and min for cross-compared locales, i.e. max(EN) - min(RU).

In [4]:
questions = range(1,25)
nationalities = ['American', 'Russian']

outlying_questions = []

for model in models:
    for question in questions:
        for nationality in nationalities:
            sliced = desc[(model)][['min','max']].loc[(nationality,question)]

            range1 = sliced.loc['EN']['max'] - sliced.loc['RU']['min']
            range2 = sliced.loc['RU']['max'] - sliced.loc['EN']['min']
            
            if range1 >= 2.0 or range2 >= 2.0:
                outlying_questions.append((model.replace('score_',''),question,nationality,max(range1,range2)))

outliers = pd.DataFrame(outlying_questions, columns=['model','question','nationality','range'])

Grouping by to get the number of occurances and list of respective questions with outliers.

In [5]:
outliers.groupby(['model', 'nationality']).agg(
    count=('range', 'count'),
    questions=('question', list)
).reset_index()

,model,nationality,count,questions
0,gpt-3.5-turbo,American,6,"[7, 12, 14, 15, 23, 24]"
1,gpt-3.5-turbo,Russian,8,"[3, 6, 7, 12, 14, 16, 21, 23]"
2,gpt-4o,Russian,4,"[7, 13, 21, 24]"
3,yandexgpt,American,4,"[12, 19, 23, 24]"
4,yandexgpt,Russian,4,"[2, 9, 19, 23]"
